In [96]:
!pip install zeyrek

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 931.0/931.0 kB 19.2 MB/s eta 0:00:00


In [88]:
pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 76.3 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=3c689fe7ef0b35328f2a267ff06d3413fa490820a189e35a4c8cabaa2b7f01b9
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [87]:
pip install JPype1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 9.5 MB/s eta 0:00:00


In [97]:
import pandas as pd
import numpy as np
import zeyrek
import nltk##lemmatization için lazım 
from jpype import JClass, getDefaultJVMPath, shutdownJVM, startJVM, JString
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import string
from sklearn.ensemble import RandomForestRegressor
import re
from transformers import AutoTokenizer, AutoModel
import torch
from sentence_transformers import SentenceTransformer
from vnlp import DependencyParser,PoSTagger,StopwordRemover, StemmerAnalyzer
#from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
#from transformers import AutoTokenizer, AutoModel
#import torch
#from vnlp import DependencyParser,PoSTagger,StopwordRemover, StemmerAnalyzer

In [ ]:
df1 = pd.read_csv("./data_result_added_mine", sep='\t')
df2 = pd.read_csv("./data_result_added", sep='\t')
frames = [df1, df2]
data = pd.concat(frames,ignore_index=True)


In [ ]:
from vnlp import DependencyParser,PoSTagger,StopwordRemover, StemmerAnalyzer

In [98]:
translator = str.maketrans('', '', string.punctuation)
nltk.download('punkt')
stopword_remover = StopwordRemover()
analyzer = zeyrek.MorphAnalyzer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [99]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

tokenizer = AutoTokenizer.from_pretrained('emrecan/bert-base-turkish-cased-mean-nli-stsb-tr')
model = AutoModel.from_pretrained('emrecan/bert-base-turkish-cased-mean-nli-stsb-tr')
def giveEmbeddings(sentences):
  encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
  with torch.no_grad():
      model_output = model(**encoded_input)
  sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
  return sentence_embeddings


In [100]:
def title_similarity():
    data["result"]=0.0
    for i in range(len(data.sentence)):
        title=stopword_remover.drop_stop_words(nltk.word_tokenize(data.loc[i,"title"].translate(translator).replace("“","").replace("’","")))
        embeddings=giveEmbeddings([data.loc[i,"sentence"],data.loc[i,"title"]])
        similarity=cosine_similarity([embeddings[0].numpy() ], [embeddings[1].numpy()])
        data.loc[i,"result"]=similarity[0][0]


In [ ]:
title_similarity()

In [ ]:
import pandas as pd

In [ ]:
data.to_csv("last_data",sep="\t",index=False)


In [18]:
data = pd.read_csv("./last_data", sep='\t')

In [26]:
data

,news_id,sentence,title,is_contains_idiom,NER,tfidf,noun_phrase,words2,sentence_length,position,result
0,144.0,"Ancak 1990’larından sonundan itibaren Türkiye,...",Dünyanın yükselen değeri Türkiye,-1,5,1,0,"['Ancak', '1990', '’', 'larından', 'sonundan',...",0.068528,0.200000,0.278959
1,144.0,Bugün artık Batı değerlerinin belirlediği ulus...,Dünyanın yükselen değeri Türkiye,-1,2,0,1,"['Bugün', 'artık', 'Batı', 'değerlerinin', 'be...",0.025381,0.166667,0.110602
2,144.0,Birçok devlet için serseri devlet kavramı artı...,Dünyanın yükselen değeri Türkiye,-1,1,0,1,"['Birçok', 'devlet', 'için', 'serseri', 'devle...",0.040609,0.142857,0.076235
3,144.0,"HAVUÇ VE SOPA POLİTİKASI BİTTİ Newsweek, eski ...",Dünyanın yükselen değeri Türkiye,-1,3,1,0,"['HAVUÇ', 'VE', 'SOPA', 'POLİTİKASI', 'BİTTİ',...",0.073604,0.125000,0.054456
4,144.0,Ama bu katkıyı verirken Amerika’nın başı çekme...,Dünyanın yükselen değeri Türkiye,-1,1,1,0,"['Ama', 'bu', 'katkıyı', 'verirken', 'Amerika'...",0.045685,0.111111,0.047487
...,...,...,...,...,...,...,...,...,...,...,...
18886,1697.0,"Galatasaray, Gaziantepspor maçının hazırlıklar...",Floryada Lucas Neill rüzgarı,-1,2,0,0,"['Galatasaray', 'Gaziantepspor', 'maçının', 'h...",0.042254,0.076923,0.073290
18887,1698.0,"Başbakan Recep Tayyip Erdoğan, özel bir haber ...",Erdoğan bir dava daha kazandı,-1,3,1,3,"['Başbakan', 'Recep', 'Tayyip', 'Erdoğan', 'öz...",0.200000,1.000000,0.404898
18888,1698.0,Ankara 3.,Erdoğan bir dava daha kazandı,-1,0,0,0,"['Ankara', '3']",0.013793,0.500000,0.271632
18889,1698.0,Asliye Hukuk Mahkemesi'nde görülen davanın kar...,Erdoğan bir dava daha kazandı,-1,5,1,2,"['Asliye', 'Hukuk', 'Mahkemesinde', 'görülen',...",0.117241,0.333333,0.568497


In [110]:
print(len(data))
import zipfile

18904


In [108]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [111]:
zip_ref = zipfile.ZipFile("/content/drive/My Drive/nlp/zip.zip", 'r')
zip_ref.extractall("/content")
zip_ref.close()

In [20]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.svm import SVR
import pickle
import math
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
import inspect

In [24]:
data=data.dropna()
data.isnull().sum()
data.reset_index(drop=True, inplace=True)

In [60]:
import random

# Initialize an empty set to store unique numbers
unique_numbers = set()
uniques=data.news_id.unique() 
# Generate random numbers until we have 2000 unique numbers
while len(unique_numbers) < 852:
    # Generate a random number between 0 and 2500
    random_number = random.randint(0, len(uniques)-1)
    # Add the number to the set if it's not already in it
    unique_numbers.add(uniques[random_number])

# Convert the set to a list and sort it
unique_numbers_list = sorted(list(unique_numbers))

In [61]:
X_rand_train=[]
X_rand_test=[]
for i in range(len(data)):
  if data.loc[i,"news_id"] in unique_numbers_list:
    X_rand_train.append(data.loc[i,:])
  else:
    X_rand_test.append(data.loc[i,:])

In [ ]:
len(X_rand_train)

15023

In [ ]:
len(X_rand_test)

3077

In [62]:
random_train = pd.DataFrame (X_rand_train, columns = data.columns)
random_test = pd.DataFrame (X_rand_test, columns = data.columns)

In [63]:
X_tr=random_train.loc[:,["news_id","sentence","title","NER","noun_phrase","tfidf","sentence_length","position"]]
X_ts=random_test.loc[:,["news_id","sentence","title","NER","noun_phrase","tfidf","sentence_length","position"]]
y_train=random_train.loc[:,["result"]]
y_test=random_test.loc[:,["result"]]

In [64]:
X_train=X_tr.loc[:,["NER","noun_phrase","tfidf","sentence_length","position"]]
X_test=X_ts.loc[:,["NER","noun_phrase","tfidf","sentence_length","position"]]

In [65]:
regressor = RandomForestRegressor(n_estimators = 600, random_state = 42)
regressor.fit(X_train,y_train) 

<ipython-input-65-9f03bb6509aa>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regressor.fit(X_train,y_train)


RandomForestRegressor(n_estimators=600, random_state=42)

In [133]:
# path to jar file 
ZEMBEREK_PATH = r"./zip/zemberek-full.jar"
# start jvm
startJVM(getDefaultJVMPath(), "-ea", "-Djava.class.path=%s" % (ZEMBEREK_PATH),convertStrings=False)
TurkishMorphology: JClass = JClass("zemberek.morphology.TurkishMorphology")
PerceptronNer: JClass = JClass("zemberek.ner.PerceptronNer")
Paths: JClass = JClass("java.nio.file.Paths")
modelRoot = Paths.get('zip/test-model/model')
morphology = TurkishMorphology.createWithDefaults()
ner = PerceptronNer.loadModel(modelRoot, morphology)

OSError: ignored

In [66]:
Y_pred = regressor.predict(X_test)

In [67]:
m1=mean_squared_error(y_test,Y_pred)
m2=mean_absolute_error(y_test,Y_pred)
r2=r2_score(y_test,Y_pred, force_finite=False)
print(m1,m2,r2)

0.026706858624689658 0.1273855112736133 0.13004233126732312


In [68]:
logr  = LinearRegression()
logr.fit(X_train,y_train) 

LinearRegression()

In [69]:
y_pred3= logr.predict(X_test)
mL1=mean_squared_error(y_test,y_pred3)
mL2=mean_absolute_error(y_test,y_pred3)
r2L=r2_score(y_test,y_pred3, force_finite=False)
print(mL1,mL2,r2L)

0.02491782843422872 0.12552475343028147 0.18831876713188977


In [70]:
X_ts.reset_index(drop=True, inplace=True)
X_ts

,news_id,sentence,title,NER,noun_phrase,tfidf,sentence_length,position
0,145.0,Antalya 12 Şubat 2008'den itibaren şehrin deği...,MOBESEler suç oranını düşürüyor,1,4,0,0.049057,1.000000
1,145.0,"158'i hareketli, 337 tanesi sabit olan kameral...",MOBESEler suç oranını düşürüyor,1,2,0,0.045283,0.500000
2,145.0,Kameraların aktardığı görüntüler mobese merkez...,MOBESEler suç oranını düşürüyor,0,2,0,0.033962,0.333333
3,145.0,Güvenlik güçleri gördükleri en ufak olumsuzluk...,MOBESEler suç oranını düşürüyor,0,2,0,0.045283,0.250000
4,145.0,Suçluların tespit edilmesinde kameralar polisi...,MOBESEler suç oranını düşürüyor,0,0,0,0.033962,0.200000
...,...,...,...,...,...,...,...,...
3753,1695.0,Manchester derbisinde 1996'dan bu yana gol ata...,Tevezin intikamı korkunç oldu,2,0,1,0.090426,0.200000
3754,1695.0,Evsahibi taraftarları sevindiren gol ise Rafae...,Tevezin intikamı korkunç oldu,3,0,1,0.122340,0.166667
3755,1695.0,"Tevez, takımını öne geçiren golünü ise 65. dak...",Tevezin intikamı korkunç oldu,0,0,1,0.058511,0.142857
3756,1695.0,2-1 yenik duruma düşmesinin ardından rakibini ...,Tevezin intikamı korkunç oldu,3,0,1,0.164894,0.125000


In [71]:
def getMaxs(X,Y,Y_real):
  Y_real=Y_real.to_numpy()
  max_values = []
  real_max_values=[]
  index=-1
  real_index=-1
  curr_id=0
  max=-1
  real_max=-1
  for i in range(len(X.loc[:,"news_id"])):
    if curr_id != X.loc[i,"news_id"].astype(int):
      curr_id=X.loc[i,"news_id"].astype(int)
      if index != -1:
        max_values.append(X.loc[index,:])
        real_max_values.append(X.loc[real_index,:])
      max=-1
      index=-1
      real_max=-1
      real_index=-1
    if Y[i]>max:
      max=Y[i]
      index=i
    if Y_real[i][0]>real_max:
      real_max=Y_real[i][0]
      real_index=i
  return (max_values,real_max_values)
    


In [72]:
(b,r_b)=getMaxs(X_ts,Y_pred,y_test)

In [84]:
a=b[:2]

In [74]:
summed=0
for i in range(len(b)):
  if b[i]["sentence"]==r_b[i]["sentence"]:
    summed+=1

In [79]:
summed/len(b)

0.27358490566037735

In [80]:
from vnlp import DependencyParser,PoSTagger,StopwordRemover, StemmerAnalyzer

In [86]:
deps_parser=dependency_parser()

In [ ]:
deps_parser

[[(1, "Erzincan'da", 16, 'obl'),
  (2, 'dindar', 3, 'amod'),
  (3, 'vatandaşlara', 6, 'nmod'),
  (4, 'yönelik', 3, 'case'),
  (5, 'hukuksuz', 6, 'amod'),
  (6, 'gözaltılar', 7, 'nmod:poss'),
  (7, 'sebebiyle', 16, 'obl'),
  (8, 'savcılığın', 9, 'nmod:poss'),
  (9, 'ifadesini', 10, 'obj'),
  (10, 'almak', 11, 'xcomp'),
  (11, 'istemesine', 15, 'advcl'),
  (12, 'rağmen', 11, 'case'),
  (13, 'günlerdir', 15, 'advmod'),
  (14, 'ifadeye', 15, 'obl'),
  (15, 'gitmeyen', 0, 'root'),
  (16, ';', 28, 'punct'),
  (17, 'bu', 19, 'nsubj'),
  (18, 'da', 17, 'advmod:emph'),
  (19, 'yetmezmiş', 28, 'advcl'),
  (20, 'gibi', 19, 'case'),
  (21, 'önceki', 22, 'amod'),
  (22, 'gün', 28, 'obl'),
  (23, 'de', 22, 'advmod:emph'),
  (24, 'askeri', 25, 'amod'),
  (25, 'araçları', 27, 'obj'),
  (26, 'sokağa', 27, 'obl'),
  (27, 'döken', 26, 'acl'),
  (28, '3', 17, 'nummod'),
  (29, '.', 28, 'punct')],
 [(1, 'Adalet', 2, 'nmod:poss'),
  (2, 'Bakanlığı', 22, 'nsubj'),
  (3, ',', 2, 'punct'),
  (4, 'askerlere', 8

In [ ]:
import pickle
with open("test4", "wb") as fp:   #Pickling
  pickle.dump(depsss, fp)


In [ ]:
basliks=[]
for title in titles:
  baslik=""
  if type(title)==str:
    basliks.append(title)
    continue
  for kelime in title:
    baslik +=" "+kelime[1]
  basliks.append(baslik)



In [ ]:
basliks

[' gitmeyen ifadeye',
 ' taslağı Bakanlığı iptalinden kanun bir hazırladı yasanın teklifi Adalet',
 ' olduğunu vurguladı',
 " aldığından Cem emin kendinden Ahmet Cübbeli Hoca Yılmaz'ın",
 ' " başta araçlarda dedi Aydın ÇİN DİKKAT parçaları MALINA',
 ' ortaya olduğu çıkmış',
 ' balans sözlerini demokrasiye yapılması ayarı hatırlattı',
 ' yeniden üniversiteye Merkezi değiştirecek',
 ' çarptı )',
 '']

In [214]:
sentence="""Salı günü yayınlanan bir röportajda Krishna, şirketin insan kaynakları gibi arka ofis işlevleri için işe alımları yavaşlatmayı veya tamamen askıya almayı planladığını açıkladı ve bu müşteriyle doğrudan karşı karşıya olmayan rollerin şu anda yaklaşık 26 bin çalışana ulaştığına dikkat çekti. Krishna , "Beş yıllık bir süre içinde bunun yüzde 30'unun yapay zeka ve otomasyonla değiştirildiğini kolayca görebiliyorum" dedi ve yapay zeka kaynaklı işten çıkarmaların yaklaşık 7 bin 800 işçiyi etkileyebileceğini öne sürdü. Bir IBM sözcüsü, şirketin şu anda bu pozisyonları dolduran herhangi bir kişiyi kovma niyetinde olmadığını, ancak yıpranma nedeniyle boşalan işlerin artık yeniden doldurulmayacağına dikkat çekti. Teknoloji devinin başka bir sözcüsü de genel bir işe alma 'duraklatma' uygulamasının olmadığını ve IBM'in gelir getirici rollere odaklanarak işe alımlarda düşünceli davrandığını açıkladı. Sözcü, şirketin "müşterilerimize veya teknolojimize doğrudan temas etmeyen işleri doldururken çok seçici" olduğunu sözlerine ekledi. IBM'in hala binlerce pozisyon için aktif olarak işe aldığına dikkat çekiyor"""

In [215]:
d = {'news_id': [], 'sentence': [],"words":[],"title":[]}
data = pd.DataFrame(data=d)
data

,news_id,sentence,words,title


In [130]:
translator = str.maketrans('', '', string.punctuation)
stopword_remover = StopwordRemover()

In [216]:
arr=sent_tokenize(sentence)
for j in arr:
    dicta = {'news_id':0,'sentence': j,"words":[nltk.word_tokenize(j.translate(translator).replace("“","").replace("’","").replace("'",""))]}
    result = pd.DataFrame(dicta)
    data = pd.concat([data,result],ignore_index=True)

In [217]:
data

,news_id,sentence,words,title
0,0.0,"Salı günü yayınlanan bir röportajda Krishna, ş...","[Salı, günü, yayınlanan, bir, röportajda, Kris...",NaN
1,0.0,"Krishna , ""Beş yıllık bir süre içinde bunun yü...","[Krishna, Beş, yıllık, bir, süre, içinde, bunu...",NaN
2,0.0,"Bir IBM sözcüsü, şirketin şu anda bu pozisyonl...","[Bir, IBM, sözcüsü, şirketin, şu, anda, bu, po...",NaN
3,0.0,Teknoloji devinin başka bir sözcüsü de genel b...,"[Teknoloji, devinin, başka, bir, sözcüsü, de, ...",NaN
4,0.0,IBM'in hala binlerce pozisyon için aktif olara...,"[IBMin, hala, binlerce, pozisyon, için, aktif,...",NaN


In [218]:
def entities_in_string(a_string):
    result = ner.findNamedEntities(JString(str(a_string.replace('i', 'İ').upper())))
    namedEntities = result.getNamedEntities();
    return namedEntities#nerin fonksiyon hali

In [242]:
data["NER"]=0
t=0;
for i in data.sentence:
    a=entities_in_string(i)
    data.loc[t,"NER"]=len(a)
    t+=1

In [220]:
def tfidf(thres):
    data["tfidf"]=0
    ind=0
    ind2=0
    while ind<len(data.sentence):
        id=data.loc[ind,"news_id"]
        metin=ret_text(ind)
        vectorizer = TfidfVectorizer()
        X = vectorizer.fit_transform([metin])
        X.toarray()
        tfidf_dict = dict(zip(vectorizer.get_feature_names_out(), X.toarray().sum(axis=0)))
        arr=[]
        print("works")
        for word in  tfidf_dict:
            if tfidf_dict[word]>thres:
                arr.append(word)
        #print(arr)
        while ind<len(data.news_id)and ind2==data.loc[ind,"news_id"]:
            for word in arr:
                if word in data.loc[ind,"sentence"]:
                    data.loc[ind,"tfidf"]+=1
            ind+=1
        
        ind2+=1

In [221]:
def ret_text(ind):
    id=data.loc[ind,"news_id"]
    metin=""
    while ind<len(data.news_id) and data.loc[ind,"news_id"]==id:
        metin+=data.loc[ind,"sentence"]
        ind+=1
    return metin

In [243]:
tfidf(0.2)

works


In [223]:
dep_parser = DependencyParser()
def dependency_parser():
    data["noun_phrase"]=0
    i=0
    for sentence in data.sentence:
        #print(sentence)
        try:
            parsed=dep_parser.predict(sentence)
            for token in parsed:
                if(token[3]=="nmod"):
                    data.loc[i,"noun_phrase"]+=1
        except Exception:
            a=sentence.split()
            parsed=dep_parser.predict(" ".join(a[0:len(a)//2]))
            for token in parsed:
                if(token[3]=="nmod"):
                    data.loc[i,"noun_phrase"]+=1
            parsed=dep_parser.predict(" ".join(a[len(a)//2:len(a)]))
            for token in parsed:
                if(token[3]=="nmod"):
                    data.loc[i,"noun_phrase"]+=1
        
        i+=1
                

In [229]:
dependency_parser()

In [244]:
data

,news_id,sentence,words,title,NER,tfidf,sentence_length,position,noun_phrase
0,0.0,"Salı günü yayınlanan bir röportajda Krishna, ş...","[Salı, günü, yayınlanan, bir, röportajda, Kris...",NaN,0,3,40,1.000000,5
1,0.0,"Krishna , ""Beş yıllık bir süre içinde bunun yü...","[Krishna, Beş, yıllık, bir, süre, içinde, bunu...",NaN,0,2,31,0.500000,0
2,0.0,"Bir IBM sözcüsü, şirketin şu anda bu pozisyonl...","[Bir, IBM, sözcüsü, şirketin, şu, anda, bu, po...",NaN,0,1,25,0.333333,0
3,0.0,Teknoloji devinin başka bir sözcüsü de genel b...,"[Teknoloji, devinin, başka, bir, sözcüsü, de, ...",NaN,1,3,39,0.250000,0
4,0.0,IBM'in hala binlerce pozisyon için aktif olara...,"[IBMin, hala, binlerce, pozisyon, için, aktif,...",NaN,0,1,11,0.200000,0


In [245]:
def add_length():
    data["sentence_length"]=0
    for i in range(len(data.words)):
        data.loc[i,"sentence_length"]=len(data.loc[i,"words"])

In [246]:
add_length()

In [247]:
def add_sentence_position():
    data["position"]=0.0
    ind=0
    rate=1
    metin_ind=0
    while ind<len(data.news_id):
        if(data.loc[ind,"news_id"]==metin_ind):
            data.loc[ind,"position"]=1/rate
            rate+=1
            ind+=1
        else:
            rate=1
            metin_ind+=1

In [248]:
add_sentence_position()

In [249]:
data

,news_id,sentence,words,title,NER,tfidf,sentence_length,position,noun_phrase
0,0.0,"Salı günü yayınlanan bir röportajda Krishna, ş...","[Salı, günü, yayınlanan, bir, röportajda, Kris...",NaN,0,3,40,1.000000,5
1,0.0,"Krishna , ""Beş yıllık bir süre içinde bunun yü...","[Krishna, Beş, yıllık, bir, süre, içinde, bunu...",NaN,0,2,31,0.500000,0
2,0.0,"Bir IBM sözcüsü, şirketin şu anda bu pozisyonl...","[Bir, IBM, sözcüsü, şirketin, şu, anda, bu, po...",NaN,0,1,25,0.333333,0
3,0.0,Teknoloji devinin başka bir sözcüsü de genel b...,"[Teknoloji, devinin, başka, bir, sözcüsü, de, ...",NaN,1,3,39,0.250000,0
4,0.0,IBM'in hala binlerce pozisyon için aktif olara...,"[IBMin, hala, binlerce, pozisyon, için, aktif,...",NaN,0,1,11,0.200000,0


In [250]:
X=data
Y_pred = regressor.predict(data.loc[:,["NER","noun_phrase","tfidf","sentence_length","position"]])

In [251]:
Y_pred

array([0.34191711, 0.29275893, 0.34866282, 0.26404607, 0.30825319])

In [252]:
def getMaxs(X,Y):
  max_values = []
  real_max_values=[]
  index=0
  real_index=0
  curr_id=0
  max=0
  real_max=0
  for i in range(len(X.loc[:,"news_id"])):
    if Y[i]>max:
      max=Y[i]
      index=i
  max_values.append(X.loc[index,:])
  real_max_values.append(X.loc[real_index,:])
  return (max_values,real_max_values)

In [253]:
Y_pred

array([0.34191711, 0.29275893, 0.34866282, 0.26404607, 0.30825319])

In [254]:
(b,r_b)=getMaxs(X,Y_pred)

In [255]:
b

[news_id                                                          0.0
 sentence           Bir IBM sözcüsü, şirketin şu anda bu pozisyonl...
 words              [Bir, IBM, sözcüsü, şirketin, şu, anda, bu, po...
 title                                                            NaN
 NER                                                                0
 tfidf                                                              1
 sentence_length                                                   25
 position                                                    0.333333
 noun_phrase                                                        0
 Name: 2, dtype: object]

In [256]:
dep_parser = DependencyParser()
def dependency_parser():
    deps=[]
    i=0
    
    for sentence in b:
        #print(sentence)
        try:
            parsed=dep_parser.predict(sentence["sentence"])
            deps.append(parsed)
        except Exception:
          try:
            h=sentence["sentence"].split()
            parsed=dep_parser.predict(" ".join(h[0:len(h)//2]))
            parsed+=dep_parser.predict(" ".join(h[len(h)//2:len(h)]))
            deps.append(parsed)
          except Exception:
            deps.append(sentence["sentence"])
            continue
        i+=1
    return deps

In [257]:
deps_parser=dependency_parser()

In [271]:
deps_parser

[[(1, 'Bir', 3, 'det'),
  (2, 'IBM', 3, 'nmod:poss'),
  (3, 'sözcüsü', 24, 'nsubj'),
  (4, ',', 3, 'punct'),
  (5, 'şirketin', 12, 'nsubj'),
  (6, 'şu', 7, 'det'),
  (7, 'anda', 10, 'obl'),
  (8, 'bu', 9, 'det'),
  (9, 'pozisyonları', 10, 'obj'),
  (10, 'dolduran', 12, 'acl'),
  (11, 'herhangi', 12, 'det'),
  (12, 'bir', 13, 'det'),
  (13, 'kişiyi', 16, 'obj'),
  (14, 'kovma', 15, 'nmod:poss'),
  (15, 'niyetinde', 16, 'obl'),
  (16, 'olmadığını', 24, 'ccomp'),
  (17, ',', 24, 'punct'),
  (18, 'ancak', 21, 'cc'),
  (19, 'yıpranma', 20, 'nmod:poss'),
  (20, 'nedeniyle', 21, 'obl'),
  (21, 'boşalan', 22, 'acl'),
  (22, 'işlerin', 24, 'nsubj'),
  (23, 'artık', 24, 'advmod'),
  (24, 'yeniden', 26, 'advmod'),
  (25, 'doldurulmayacağına', 24, 'ccomp'),
  (26, 'dikkat', 0, 'root'),
  (27, 'çekti', 26, 'compound'),
  (28, '.', 26, 'punct')]]

In [270]:
titles=[]
for deps_pars in deps_parser:
  root=0
  zero=0
  title = set()
  indexes=set()
  deps_pars
  if type(deps_pars[0])==str:
    titles.append("şok şok şok")
    continue
  for j in range(len(deps_pars)):
    i=deps_pars[j]
    if i[2]==0:
      zero=i
      indexes.add(i[0])
      title.add(i)
    if i[3]=="root":
      root=i
      indexes.add(i[0])
      title.add(i)
  
  for j in range(len(deps_pars)):
    i=deps_pars[j]
    if (root!=0 and i[0]+1==root[0]) or (zero!=0 and  i[0]+1==zero[0]):
      if i[2]!="punct":
        title.add(i)
        indexes.add(i[0])

    if  (root!=0 and (i[0]-1==root[0] or i[0]+1==root[0]) and i[3]=="conj") or (zero!=0 and (i[0]-1==zero[0] or i[0]+1==zero[0]) and i[3]=="conj"):
      if i[2]!="punct":
        title.add(i)
        indexes.add(i[0])

    if ((root!=0 and i[2]==root[0] ) or (zero!=0 and i[2]==zero[0]) ) and (i[3]=="nsubj" or i[3]=="compound" or i[3]=="flat"   ):
        if i[2]!="punct":
          title.add(i)
          indexes.add(i[0])

  for i in deps_pars:
    if i[2] in indexes and (i[3]=="flat" or i[3]=="amod" or i[3]=="nmod" or i[3]=="nmod:poss" or i[3]=="csubj"or i[3]=="nsubj" or i[3]=="obl" or i[3]=="compound" or i[3]=="det" or i[3]=="nummod"or i[3]=="advmod"):
      if i[2]!="punct":
        title.add(i)
        indexes.add(i[0])

  for i in deps_pars:
    if i[2] in indexes and (i[3]=="flat" or i[3]=="amod" or i[3]=="nmod" or i[3]=="nmod:poss" or i[3]=="csubj"or i[3]=="nsubj" or i[3]=="obl" or i[3]=="compound" or i[3]=="det" or i[3]=="nummod"or i[3]=="advmod"):
      if i[2]!="punct":
        title.add(i)
        indexes.add(i[0])

  for i in deps_pars:
    if i[2] in indexes and (i[3]=="flat" or i[3]=="amod" or i[3]=="nmod" or i[3]=="nmod:poss" or i[3]=="csubj"or i[3]=="nsubj" or i[3]=="obl" or i[3]=="compound" or i[3]=="det" or i[3]=="nummod"or i[3]=="advmod"):
      if i[2]!="punct":
        title.add(i)
        indexes.add(i[0])
  if(len(title)==0 or title==None):
    title="şok şok şok"
  titles.append(title)




In [268]:
titles

[{(1, 'Bir', 3, 'det'),
  (2, 'IBM', 3, 'nmod:poss'),
  (3, 'sözcüsü', 24, 'nsubj'),
  (22, 'işlerin', 24, 'nsubj'),
  (23, 'artık', 24, 'advmod'),
  (24, 'yeniden', 26, 'advmod'),
  (25, 'doldurulmayacağına', 24, 'ccomp'),
  (26, 'dikkat', 0, 'root'),
  (27, 'çekti', 26, 'compound')}]

In [ ]:
def title_similarity():
    for i in range(len(data.sentence)):
        title=stopword_remover.drop_stop_words(nltk.word_tokenize(data.loc[i,"title"].translate(translator).replace("“","").replace("’","")))
        embeddings=giveEmbeddings([data.loc[i,"sentence"],data.loc[i,"title"]])
        similarity=cosine_similarity([embeddings[0].numpy() ], [embeddings[1].numpy()])
        data.loc[i,"result"]=similarity[0][0]

In [ ]:
toplam=0
for f in range(10):
  embeddings=giveEmbeddings([basliks[f],predicted_s[f]])
  similarity=cosine_similarity([embeddings[0].numpy()], [embeddings[1].numpy()])
  print(similarity)     

In [ ]:
toplam/10

array([[0.31325048]], dtype=float32)

In [ ]:
toplam=0
for f in range(10):
  embeddings=giveEmbeddings([a[f]["title"],predicted_s[f]])
  similarity=cosine_similarity([embeddings[0].numpy()], [embeddings[1].numpy()])
  toplam+=  similarity  